# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd

In [3]:
from sqlalchemy import create_engine

In [4]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)

In [5]:
df.head(3)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [7]:
features = ['message','genre']

In [8]:
X = df[features]
X.head(3)

,message,genre
0,Weather update - a cold front from Cuba that c...,direct
1,Is the Hurricane over or is it not over,direct
2,Looking for someone but no name,direct


In [9]:
non_target_cols = ['id','message','original','genre']
non_target_cols

['id', 'message', 'original', 'genre']

In [10]:
target_cols = df.columns[~df.columns.isin(non_target_cols)]
target_cols

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [11]:
Y = df[target_cols]
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
Y.request.value_counts()

0    21742
1     4474
Name: request, dtype: int64

### 2. Write a tokenization function to process your text data

In [46]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [47]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [48]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize) ),
    ('tfidf', TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [20]:
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, Y_train, Y_test = train_test_split(X.message,Y)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [22]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [23]:
from sklearn.multioutput import MultiOutputClassifier

In [53]:
clf = pipeline.fit(X_train, Y_train)

In [54]:
Y_pred = pipeline.predict(X_test)

In [ ]:
(Y_test.shape, Y_pred.shape)

In [ ]:
[type(Y_test), type(Y_pred)]

In [58]:
Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
Y_pred.head(2)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
import numpy as np

In [62]:
from sklearn.utils.multiclass import type_of_target

In [66]:
[type_of_target(Y_test), type_of_target(Y_pred)]

['multiclass-multioutput', 'multiclass-multioutput']

In [69]:
print(classification_report(np.hstack(Y_test), np.hstack(Y_pred),target_names=Y_test.columns))

                        precision    recall  f1-score   support

               related       1.00      1.00      1.00         1
               request       1.00      1.00      1.00         1
                 offer       1.00      1.00      1.00         1
           aid_related       1.00      1.00      1.00         1
          medical_help       1.00      1.00      1.00         1
      medical_products       1.00      1.00      1.00         1
     search_and_rescue       1.00      1.00      1.00         1
              security       1.00      1.00      1.00         1
              military       1.00      1.00      1.00         1
           child_alone       1.00      1.00      1.00         1
                 water       1.00      1.00      1.00         1
                  food       1.00      1.00      1.00         1
               shelter       1.00      1.00      1.00         1
              clothing       1.00      1.00      1.00         1
                 money       1.00      

In [72]:
print(f1_score(np.hstack(Y_test), np.hstack(Y_pred),average='macro'))

1.0


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.